### Abstract
- 前回までは、ロジスティック回帰なので、まだDeep Learningではなかった。
- ここからは、隠れ層や活性化関数を使うことで、Deep Learningになる。
- テーマはmnistをつかう。

### Imports

In [31]:
import pathlib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split

### IO

- 入出力ファイルの場所は先頭に記述する方が、後からわかりやすい。

In [5]:
INPUT_DOWNLOAD_PATH = pathlib.Path('./data').joinpath('03_mnist', 'input')

# pathlib.Path('./data').joinpath('03_mnist', 'output').mkdir(parents=True, exist_ok=True)
# OUTPUT_MODEL_FILE = pathlib.Path('./data').joinpath('03_mnist', 'output', 'mnist-logistic.pth')

### Dataset

In [16]:
train_ds = MNIST(root=INPUT_DOWNLOAD_PATH, 
                train=True,
                transform=transforms.ToTensor())

test_ds = MNIST(root=INPUT_DOWNLOAD_PATH, 
                train=False,
                transform=transforms.ToTensor())

train_ds, val_ds = random_split(train_ds, [50000, 10000])
image, label = train_ds[0]

display(f"type: {type(image)}, dtype: {image.dtype}, shape: {image.shape}")
display(f"type: {type(label)}")

"type: <class 'torch.Tensor'>, dtype: torch.float32, shape: torch.Size([1, 28, 28])"

"type: <class 'int'>"

### Config

In [11]:
BATCH_SIZE = 128

### DataLoader

In [14]:
train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, BATCH_SIZE)
test_loader = DataLoader(test_ds, BATCH_SIZE)

### Model
- 説明が面倒なので、モデル定義からやっちゃいます。
- linearが2個になっただけなんです。

In [26]:
class CustomModel(nn.Module):
    def __init__(self, input_shape, hidden_size, num_classes):
        super().__init__()
        
        input_size = 1
        for i in input_shape:
            input_size = input_size * i

        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_classes)
        
        self.input_size = input_size
        
    def forward(self, xb):
        xb = xb.reshape(-1, self.input_size)
        
        out = self.linear1(xb)
        out = F.relu(out)
        out = self.linear2(out)
        return out

model = CustomModel(image.shape, 32, 10)

- parameterのダンプは、state_dictを使った方が、名前もわかっていい。

In [27]:
for k, v in model.state_dict().items():
    print(k)
    display(f"dtype: {v.dtype}, shape: {v.shape}")

linear1.weight


'dtype: torch.float32, shape: torch.Size([32, 784])'

linear1.bias


'dtype: torch.float32, shape: torch.Size([32])'

linear2.weight


'dtype: torch.float32, shape: torch.Size([10, 32])'

linear2.bias


'dtype: torch.float32, shape: torch.Size([10])'

- 一応推論テスト

In [29]:
for images, labels in train_loader:
    outputs = model(images)
    display(f"dtype: {outputs.dtype}, shape: {outputs.shape}")
    
    loss = F.cross_entropy(outputs, labels)
    print('Loss:', loss.item())
    break

'dtype: torch.float32, shape: torch.Size([128, 10])'

Loss: 2.3086531162261963


### GPU使用
- GPUはなんかすごい並列処理できるコア
- CPUよりは、1コア当たりは貧弱。
- 元々画像処理向けなので、画像は画素毎に同じ処理をするので並列処理が得意。
- これを画像以外の用途につかうのが、GP(General Purpose)GPU。
- PyTorhcなどのライブラリはGPGPUが手軽にできるのも利点。
<br>
<br>
- デバイス数や使用可能かどうかを確かめる。

In [36]:
display(f"GPU available: {torch.cuda.is_available()}, GPU Num: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    display(f"cuda:{i}, name:{torch.cuda.get_device_name(i)}")

'GPU available: False, GPU Num: 0'

- マシンに応じた設定になるような関数

In [38]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()
device

device(type='cpu')

- 再帰呼び出しで、データを転送する関数。
- 転送自体は、tensorのtoメソッドでできるみたい。
- ちなみに、non_blockingは高速化の仕組みのようなもの。詳細はここら辺参照。
  - https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587

In [41]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [45]:
t = torch.randn(3,3,3)
to_device([t, t], device)

[tensor([[[ 0.4474,  1.9290,  0.4737],
          [-1.5279, -0.9231,  0.4110],
          [-2.0824,  0.3134,  2.5996]],
 
         [[-0.6124,  0.4139,  1.0309],
          [-1.0299, -0.0945, -1.3544],
          [ 1.2997, -0.3557,  1.3776]],
 
         [[-1.6942,  0.1673,  2.2078],
          [-2.0198,  1.1477,  0.0209],
          [-0.5538, -0.4968,  0.6305]]]),
 tensor([[[ 0.4474,  1.9290,  0.4737],
          [-1.5279, -0.9231,  0.4110],
          [-2.0824,  0.3134,  2.5996]],
 
         [[-0.6124,  0.4139,  1.0309],
          [-1.0299, -0.0945, -1.3544],
          [ 1.2997, -0.3557,  1.3776]],
 
         [[-1.6942,  0.1673,  2.2078],
          [-2.0198,  1.1477,  0.0209],
          [-0.5538, -0.4968,  0.6305]]])]